In [1]:
!hdfs dfs -mkdir -p /user/cloudera/input

In [3]:
! wget https://s3.amazonaws.com/imcbucket/data/flights/2008.csv

--2020-03-16 03:29:10--  https://s3.amazonaws.com/imcbucket/data/flights/2008.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.112.181
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.112.181|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 689413344 (657M) [binary/octet-stream]
Saving to: ‘2008.csv’

2008.csv            100%[===================>] 657.48M  77.6MB/s    in 8.1s    

2020-03-16 03:29:18 (80.8 MB/s) - ‘2008.csv’ saved [689413344/689413344]



In [4]:
!hadoop fs -put 2008.csv /user/cloudera/input/

In [5]:
!hadoop fs -ls /user/cloudera/input

Found 1 items
-rw-r--r--   2 root hadoop  689413344 2020-03-16 03:29 /user/cloudera/input/2008.csv


In [7]:
!head -3 ./2008.csv

Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
2008,1,3,4,2003,1955,2211,2225,WN,335,N712SW,128,150,116,-14,8,IAD,TPA,810,4,8,0,,0,NA,NA,NA,NA,NA
2008,1,3,4,754,735,1002,1000,WN,3231,N772SW,128,145,113,2,19,IAD,TPA,810,5,10,0,,0,NA,NA,NA,NA,NA


In [10]:
airline_rdd = sc.textFile('hdfs:///user/cloudera/input/2008.csv')

In [11]:
airline_rdd.take(2)

['Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay',
 '2008,1,3,4,2003,1955,2211,2225,WN,335,N712SW,128,150,116,-14,8,IAD,TPA,810,4,8,0,,0,NA,NA,NA,NA,NA']

In [13]:
header_text = airline_rdd.first()

In [14]:
airline_noheader_rdd = airline_rdd.filter(lambda row: row != header_text)

In [15]:
airline_noheader_rdd.take(3)

['2008,1,3,4,2003,1955,2211,2225,WN,335,N712SW,128,150,116,-14,8,IAD,TPA,810,4,8,0,,0,NA,NA,NA,NA,NA',
 '2008,1,3,4,754,735,1002,1000,WN,3231,N772SW,128,145,113,2,19,IAD,TPA,810,5,10,0,,0,NA,NA,NA,NA,NA',
 '2008,1,3,4,628,620,804,750,WN,448,N428WN,96,90,76,14,8,IND,BWI,515,3,17,0,,0,NA,NA,NA,NA,NA']

In [19]:
fieldname_list = header_text.split(',')
def make_row(row):
    fieldValue_list = row.split(',')
    d = dict(zip(fieldname_list,fieldValue_list))
    return d

In [20]:
airline_rows_rdd = airline_noheader_rdd.map(make_row)

In [21]:
airline_rows_rdd.take(3)

[{'Year': '2008',
  'Month': '1',
  'DayofMonth': '3',
  'DayOfWeek': '4',
  'DepTime': '2003',
  'CRSDepTime': '1955',
  'ArrTime': '2211',
  'CRSArrTime': '2225',
  'UniqueCarrier': 'WN',
  'FlightNum': '335',
  'TailNum': 'N712SW',
  'ActualElapsedTime': '128',
  'CRSElapsedTime': '150',
  'AirTime': '116',
  'ArrDelay': '-14',
  'DepDelay': '8',
  'Origin': 'IAD',
  'Dest': 'TPA',
  'Distance': '810',
  'TaxiIn': '4',
  'TaxiOut': '8',
  'Cancelled': '0',
  'CancellationCode': '',
  'Diverted': '0',
  'CarrierDelay': 'NA',
  'WeatherDelay': 'NA',
  'NASDelay': 'NA',
  'SecurityDelay': 'NA',
  'LateAircraftDelay': 'NA'},
 {'Year': '2008',
  'Month': '1',
  'DayofMonth': '3',
  'DayOfWeek': '4',
  'DepTime': '754',
  'CRSDepTime': '735',
  'ArrTime': '1002',
  'CRSArrTime': '1000',
  'UniqueCarrier': 'WN',
  'FlightNum': '3231',
  'TailNum': 'N772SW',
  'ActualElapsedTime': '128',
  'CRSElapsedTime': '145',
  'AirTime': '113',
  'ArrDelay': '2',
  'DepDelay': '19',
  'Origin': 'IAD',

In [23]:
airline_rows_rdd_cached = airline_rows_rdd.repartition(10).cache()

In [32]:
!cp -rf /usr/lib/hive/conf/hive-site.xml /usr/lib/spark/conf

In [31]:
!cat /usr/lib/hive/conf/hive-site.xml

<?xml version="1.0" ?>
<!--
  Licensed to the Apache Software Foundation (ASF) under one or more
  contributor license agreements.  See the NOTICE file distributed with
  this work for additional information regarding copyright ownership.
  The ASF licenses this file to You under the Apache License, Version 2.0
  (the "License"); you may not use this file except in compliance with
  the License.  You may obtain a copy of the License at

      http://www.apache.org/licenses/LICENSE-2.0

  Unless required by applicable law or agreed to in writing, software
  distributed under the License is distributed on an "AS IS" BASIS,
  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
  See the License for the specific language governing permissions and
  limitations under the License.
-->
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<configuration>
  <!-- Hive Configuration can either be stored in this file or in the hadoop configuration files  -->
  <!-- that

In [33]:
airline_rows_df = airline_rows_rdd.toDF()

/usr/lib/spark/python/pyspark/sql/session.py:360: UserWarning: Using RDD of dict to inferSchema is deprecated. Use pyspark.sql.Row instead
  warnings.warn("Using RDD of dict to inferSchema is deprecated. "


In [34]:
airline_rows_df.registerTempTable("airline")

In [36]:
results = sqlContext.sql("select avg(ArrDelay) as delay, UniqueCarrier \
from airline \
group by UniqueCarrier \
order by delay ASC \
limit 10")

In [37]:
results.show()

+------------------+-------------+
|             delay|UniqueCarrier|
+------------------+-------------+
|-2.888673890608875|           AQ|
|1.2644089394236424|           HA|
|2.8481100056939694|           US|
| 4.111134703250211|           9E|
| 4.804346362093581|           AS|
|  5.17967817300539|           WN|
| 6.108246666107523|           F9|
| 6.598884736868734|           OO|
| 7.368539129929383|           NW|
| 7.855163154883384|           DL|
+------------------+-------------+



In [ ]:
sqlContext.sql("select avg(ArrDelay) as delay, UniqueCarrier \
from airline \
group by UniqueCarrier \
order by delay DESC \
limit 10").show()